In [73]:
import numpy as mp
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [74]:
df=sns.load_dataset('tips')

In [75]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [77]:
df['sex'].value_counts()

sex
Male      157
Female     87
Name: count, dtype: int64

In [78]:
df['day'].value_counts()

day
Sat     87
Sun     76
Thur    62
Fri     19
Name: count, dtype: int64

## label encoding for sex and day
## onehot encoding for others

In [79]:
df.columns

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')

In [80]:
X=df[['tip', 'sex', 'smoker', 'day', 'time', 'size']]

In [81]:
y=df['total_bill']

In [82]:
from sklearn.model_selection import train_test_split

In [83]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=42)

In [84]:
X_train

,tip,sex,smoker,day,time,size
115,3.50,Female,No,Sun,Dinner,2
181,5.65,Male,Yes,Sun,Dinner,2
225,2.50,Female,Yes,Fri,Lunch,2
68,2.01,Male,No,Sat,Dinner,2
104,4.08,Female,No,Sat,Dinner,2
...,...,...,...,...,...,...
106,4.06,Male,Yes,Sat,Dinner,2
14,3.02,Female,No,Sun,Dinner,2
92,1.00,Female,Yes,Fri,Dinner,2
179,3.55,Male,Yes,Sun,Dinner,2


## Feature encoding

In [85]:
from sklearn.preprocessing import LabelEncoder

In [86]:
le1=LabelEncoder()
le2=LabelEncoder()
le3=LabelEncoder()

In [87]:
X_train['sex']=le1.fit_transform(X_train['sex'])
X_train['smoker']=le2.fit_transform(X_train['smoker'])
X_train['time']=le3.fit_transform(X_train['time'])


In [88]:
X_train

,tip,sex,smoker,day,time,size
115,3.50,0,0,Sun,0,2
181,5.65,1,1,Sun,0,2
225,2.50,0,1,Fri,1,2
68,2.01,1,0,Sat,0,2
104,4.08,0,0,Sat,0,2
...,...,...,...,...,...,...
106,4.06,1,1,Sat,0,2
14,3.02,0,0,Sun,0,2
92,1.00,0,1,Fri,0,2
179,3.55,1,1,Sun,0,2


In [89]:
X_test['sex']=le1.transform(X_test['sex'])
X_test['smoker']=le2.transform(X_test['smoker'])
X_test['time']=le3.transform(X_test['time'])


In [90]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [91]:
ct = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(drop='first'), [3])  # Corrected to use a list for the column index
    ],
    remainder='passthrough'  # This keeps the other columns unchanged
)

In [92]:
X_train=ct.fit_transform(X_train)

In [93]:
X_test=ct.transform(X_test)

In [94]:
X_train

array([[0., 1., 0., ..., 0., 0., 2.],
       [0., 1., 0., ..., 1., 0., 2.],
       [0., 0., 0., ..., 1., 1., 2.],
       ...,
       [0., 0., 0., ..., 1., 0., 2.],
       [0., 1., 0., ..., 1., 0., 2.],
       [1., 0., 0., ..., 1., 0., 3.]])

In [95]:
from sklearn.svm import SVR

In [96]:
svr=SVR()

In [97]:
svr.fit(X_train,y_train)

SVR()

In [98]:
y_pred=svr.predict(X_test)

In [99]:
from sklearn.metrics import r2_score,mean_absolute_error
r2_score(y_test,y_pred)


0.49798620106004743

In [100]:
mean_absolute_error(y_test,y_pred)

4.463296539661225

In [101]:
from sklearn.model_selection import GridSearchCV

In [102]:
param_grid={
    'C':[0.1,1,10,100,1000],
    'gamma':[1,0.1,0.001,0.0001],
    'kernel':['rbf']
}

In [103]:
grid=GridSearchCV(SVR(),param_grid,refit=True,verbose=3)

In [104]:
grid.fit(X_train,y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 1/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.114 total time=   0.0s
[CV 2/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.008 total time=   0.0s
[CV 3/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.003 total time=   0.0s
[CV 4/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.015 total time=   0.0s
[CV 5/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.123 total time=   0.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.019 total time=   0.0s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.096 total time=   0.0s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.078 total time=   0.0s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.099 total time=   0.0s
[CV 5/5] END .....C=0.1, gamma=0.1, kernel=rbf;, score=-0.020 total time=   0.0s
[CV 1/5] END ...C=0.1, gamma=0.001, kernel=rbf;, score=-0.155 total time=   0.0s
[CV 2/5] END ...C=0.1, gamma=0.001, kernel=rbf;

GridSearchCV(estimator=SVR(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.001, 0.0001], 'kernel': ['rbf']},
             verbose=3)

In [105]:
grid.best_params_

{'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}

In [106]:
y_pred=grid.predict(X_test)

In [107]:
r2_score(y_test,y_pred)

0.591289890679547

In [108]:
mean_absolute_error(y_test,y_pred)

4.17258788487694